In [1]:
!pip install pytesseract transformers sentence-transformers faiss-cpu chromadb openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 16.9 MB/s eta 

In [4]:
!pip install langchain openai faiss-cpu tiktoken python-dotenv pypdf langchain-community langchain_openai
!pip install langchain-google-genai
!pip install groq langchain-groq

  Using cached pypdf-6.0.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_openai-0.3.30-py3-none-any.whl.metadata (2.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.1

In [ ]:
!pip install Pillow

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import numpy as np
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import pandas as pd
from google.colab import userdata
import os
from langchain_groq import ChatGroq
import re

In [13]:
groq_api_key = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = groq_api_key

In [ ]:
import pytesseract
from PIL import Image

def perform_ocr(image_path):
    """
    Performs OCR on an image file and returns the extracted text.

    Args:
        image_path (str): The path to the image file.

    Returns:
        str: The extracted text from the image.
    """
    try:
        # Open the image file
        img = Image.open(image_path)
        # Perform OCR using pytesseract
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        print(f"Error performing OCR on {image_path}: {e}")
        return ""

In [18]:
import os

# Define the directory containing the bank screenshots
bankimages_dir = 'bankimages' # Assuming the folder is in the same directory as the notebook

extracted_texts_with_paths = []
original_text_snippets = [] # Store only the text for easy retrieval

# Check if the bankimages directory exists
if not os.path.exists(bankimages_dir):
    print(f"Error: The directory '{bankimages_dir}' was not found.")
else:
    # List all files in the directory and filter for image files (e.g., .jpg, .png)
    image_files = [f for f in os.listdir(bankimages_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]

    if not image_files:
        print(f"No image files found in the directory '{bankimages_dir}'.")
    else:
        for image_file in image_files:
            image_path = os.path.join(bankimages_dir, image_file)
            extracted_text = perform_ocr(image_path) # Use the perform_ocr function
            extracted_texts_with_paths.append({'path': image_path, 'text': extracted_text})
            original_text_snippets.append(extracted_text)

        print(f"Processed {len(extracted_texts_with_paths)} images and extracted text.")
        for item in extracted_texts_with_paths:
            print(f"Path: {item['path']}, Text: {item['text'][:50]}...") # Print first 50 chars

Processed 9 images and extracted text.
Path: bankimages/Login-Step1.jpg, Text: Welcome to HDFC Bank NetBanking
le2HDFC Bank

Logi...
Path: bankimages/Login-step3.jpg, Text: More authentication needed!

We at HDFC Bank take ...
Path: bankimages/Cards.jpg, Text: ‘SmantBuy Personalize UserID Insta Alerts SMS Bank...
Path: bankimages/login-step4.jpg, Text: More authentication needed!

Please note:

4. Ther...
Path: bankimages/AccountBalance.jpg, Text: fearon

‘Account Summary

Transact >

Enquire v

‘...
Path: bankimages/Login-Step2.jpg, Text: Welcome to HDFC Bank NetBanking
| so: EEE nw

Logi...
Path: bankimages/FundTransfer.jpg, Text: ha BANK
ees

 

 

 

 

 

 

 

 

 

 

 

 

 ...
Path: bankimages/HomePage.jpg, Text: ‘SmartBuy Personalize User ID Insta Alerts SMS Ban...
Path: bankimages/AccountStatment.jpg, Text: N
fara BANK

 

  
   

 

“ View / Download Accou...


In [38]:
#Apply Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=100)
documents = [Document(page_content=item['text']) for item in extracted_texts_with_paths] # Correct: creates a Document for each text snippet
texts = text_splitter.split_documents(documents)

 #4. create Vector DB
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert embeddings to numpy array of float32 before creating FAISS index
import numpy as np
text_embeddings_np = np.array([embeddings.embed_query(doc.page_content) for doc in texts]).astype('float32')

db_img_text = FAISS.from_documents(texts, embeddings)


llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
retriever = db_img_text.as_retriever(search_kwargs={"k": 5})
#qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [42]:
def GetContext(question: str):
  # Use the retriever to get relevant documents based on the question
  docs = retriever.get_relevant_documents(question)
  # Extract the page content from the documents and join them
  context = "\n".join([doc.page_content for doc in docs])
  return context

In [39]:
faq="how to view account statement"

In [40]:
TOOL_MAPPING ={
     "GetContext":GetContext
}

In [43]:
TOOLS = [
     {
        "type": "function",
        "function":{
            "name":"GetContext",
            "description":"gets all the required context documents",
            "parameters":{
                "type":"object",
                "properties":{
                    "question":{"type":"str","description":"user question"}
                },
                "required":["question"]
            },
        },
    },
]

PROMPT = (
    f"{faq}, you can use GetContext to read all required documents data to answer"
    "Return this value **only** with no other words"
)

In [44]:
import os
from groq import Groq

try:
    # Get the Groq API key from environment variables
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY not found in environment variables.")

    # Initialize the Groq client
    client = Groq(api_key=groq_api_key)

    # Generate content using the Groq client and a Gemini model
    # Note: Replace 'gemini-model-name' with the actual name of the Gemini model available on Groq.
    # You may need to check Groq's documentation for the exact model names.
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": PROMPT,
            }
        ],
        model= "llama-3.3-70b-versatile",#"openai/gpt-oss-20b",#"GPT‑4o‑Lite", #openai/gpt-oss-120b #"llama-3.3-70b-versatile", # Example model name, replace with actual
        tools=TOOLS,
        tool_choice="auto",
    )

    # Print the response
    print(chat_completion.choices[0].message)

except Exception as e:
    print(f"An error occurred during content generation: {e}")
    #print("Please ensure you have a valid GROQ_API_KEY configured and the request is valid.")

An error occurred during content generation: Error code: 400 - {'error': {'message': 'schema is not valid JSON Schema for tool GetContext parameters: jsonschema file:///home/di/params.json compilation failed: \'/properties/question/type\' does not validate with https://json-schema.org/draft/2020-12/schema#/allOf/1/$ref/properties/properties/additionalProperties/$dynamicRef/allOf/3/$ref/properties/type/anyOf/0/$ref/enum: value must be one of "array", "boolean", "integer", "null", "number", "object", "string"', 'type': 'invalid_request_error'}}


In [36]:
tool_calls = chat_completion.choices[0].message.tool_calls
tool_messages = []
if tool_calls: # Check if tool_calls is not None
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        # It's generally safer to use a more controlled way to access arguments
        # than direct eval(). However, given the context, we'll proceed with eval
        # but acknowledge the security implications if the input is not trusted.
        args = eval(tool_call.function.arguments)
        result  = TOOL_MAPPING[func_name](**args)
        print(result)
        print(args)
        print(func_name)
        tool_messages.append({
            "role":"tool",
            "tool_call_id":tool_call.id,
            "content":str(result)
            }
        )
else:
    print("No tool calls were returned by the model.")

N
fara BANK

 

  
   

 

“ View / Download Account Statement oe
‘Account Summary
sono An AMI. 21208 coton ewan sre
‘Transact a
|[ Select Another Account / Period | Page 1 of 1
— |
Enquire ’
ae Naraten Chequemet.No. | Value Date windrawal Depost Closing
View Account Balance a
vpesmana
ic Statement - Current & un 2nas | cfstartpggecashireensapb- sss :
‘Me Siatoment zmog nas | Ssmanana@easnteensan seeraszzises — 12aup2008 12000 SS
WELLSFARGOHYDITpop
A\c Statement - Upto 5 Years: UPI-Smara-
ctsmanapo@casntenspo-
t1Aug 2008 | stanapg@casnteensapy sesesisstets — Awg202 ©1200
‘Ne Statement -Upto 10 WELLSFARGOHYD17oUt
Years

upL-smana-
ctsmangpg@cashtreensaipo-

Feed Depot Summary tvaup2oes | fatanapgacaneensgo- svcotTasrs— 11augz00s 400 Sa
Sea

Recurring Deposit Summary pian.
ctsmangpg@cashtreensaipo-

NsPa0000011-149201515087

WELLSFARGOHYDI79ru

11 aug 2025 asa01sts687 © 1Aug2025 5000

View Cheque Status
fearon

‘Account Summary

Transact >

Enquire v

‘View Account Balance

‘Nc Statement

In [37]:
response2 = client.chat.completions.create(
    model= "llama-3.3-70b-versatile",#"openai/gpt-oss-20b",#"llama-3.3-70b-versatile",
    messages=[
        {"role": "user", "content": PROMPT},
        chat_completion.choices[0].message,
        *tool_messages
    ],
    tools= TOOLS,
    tool_choice="auto"
)

print(response2.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=[ChatCompletionMessageToolCall(id='2m3hvxxj6', function=Function(arguments='{"question":"how to view account balance in HDFC bank"}', name='GetContext'), type='function')])
